In [ ]:
!git clone https://github.com/idealo/image-quality-assessment.git

Cloning into 'image-quality-assessment'...
remote: Enumerating objects: 531, done.
remote: Total 531 (delta 0), reused 0 (delta 0), pack-reused 531
Receiving objects: 100% (531/531), 79.96 MiB | 4.89 MiB/s, done.
Resolving deltas: 100% (214/214), done.


In [ ]:
import tensorflow as tf
import numpy as np
import imutils
from imutils import paths

import argparse
import cv2
from PIL import Image
import os
from google.colab.patches import cv2_imshow
from google.colab import files
import matplotlib.pyplot as plt

print(tf.__version__)

2.6.0


In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('/content/image-quality-assessment/contrib/tf_serving/tfs_models/mobilenet_technical') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('quality_assessment.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
input_image = np.asarray(tf.keras.preprocessing.image.load_img('/content/image-quality-assessment/readme_figures/images_technical/techncial3.jpg', target_size=(224,224)))
input_image = tf.keras.applications.mobilenet.preprocess_input(input_image)
input_image = tf.convert_to_tensor(input_image, dtype=tf.float32)

input_image = tf.expand_dims(input_image, axis = 0)
print(input_image.shape)

(1, 224, 224, 3)


In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/quality_assessment.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
print(input_details)
output_details = interpreter.get_output_details()
print(output_details)

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_image)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[{'name': 'input_1:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'dense_1/Softmax:0', 'index': 130, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[[0.22768664 0.14084102 0.11603771 0.08682505 0.13035013 0.06748918
  0.0558218  0.06213458 0.06899322 0.04382064]]


In [ ]:
def normalize_labels(labels):
    labels_np = np.array(labels)
    return labels_np / labels_np.sum()

def calc_mean_score(score_dist):
    score_dist = normalize_labels(score_dist)
    return (score_dist*np.arange(1, 11)).sum()

sample = calc_mean_score(output_data)

print(sample)

4.2084428407251835


# Test images

In [ ]:
!unzip /content/blur_images.zip -d /content/blur_images

In [ ]:
data = []
general_path = '/content/blur_images/'

for filename in os.listdir(general_path):
    if filename.endswith("jpg"): 
        # Your code comes here such as 
        data.append(general_path + filename)

print(data)

In [ ]:
!mkdir quality_processed_images

In [ ]:
THRESHOLD = 5.3
# loop over the input images
for imagePath in data:
  # load the image and process it
  image = cv2.imread(imagePath)

  input_image = np.asarray(tf.keras.preprocessing.image.load_img(imagePath, target_size=(224,224)))
  input_image = tf.keras.applications.mobilenet.preprocess_input(input_image)
  input_image = tf.convert_to_tensor(input_image, dtype=tf.float32)
  input_image = tf.expand_dims(input_image, axis = 0)
  
  interpreter.set_tensor(input_details[0]['index'], input_image)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])

  sample = calc_mean_score(output_data)
  
  text = "Not Blurry"
  color = (0,255,0)
  # if the focus measure is less than the supplied threshold,
  # then the image should be considered "blurry"
  if sample < THRESHOLD:
    text = "Blurry"
    color = (0,0,255)
  # show the image
  cv2.putText(image, "{}: {:.2f}".format(text, sample), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 3)
  cv2_imshow(image)
  key = cv2.waitKey(0)

  name = os.path.split(imagePath)[1]
  #print(name)
  
  #write the processed image to specific folder
  cv2.imwrite(('/content/quality_processed_images/'+ name), image)

In [ ]:
#zip folder
!zip -r /content/quality_processed_images.zip /content/quality_processed_images

## Find the center of image

In [ ]:
!mkdir cropped_quality_images

In [ ]:

def im_crop_center(img, w, h):
    img_width, img_height = img.size
    left, right = (img_width - w) / 2, (img_width + w) / 2
    top, bottom = (img_height - h) / 2, (img_height + h) / 2
    left, top = round(max(0, left)), round(max(0, top))
    right, bottom = round(min(img_width - 0, right)), round(min(img_height - 0, bottom))
    return img.crop((left, top, right, bottom))


for path in data:
  print(path)
  pil_image = Image.open(path)

  open_cv_image = np.array(im_crop_center(pil_image, 640, 480)) 
  # Convert RGB to BGR 
  open_cv_image = open_cv_image[:, :, ::-1].copy()
  name = os.path.split(path)[1]
  cv2.imwrite(('/content/cropped_quality_images/'+ name), open_cv_image)
  #cv2_imshow(open_cv_image)

In [ ]:
dataCropped = []
general_path_cropped = '/content/cropped_quality_images/'

for filename in os.listdir(general_path_cropped):
    if filename.endswith("jpg"): 
        # Your code comes here such as 
        dataCropped.append(general_path_cropped + filename)

print(dataCropped)

In [ ]:
!mkdir quality_cropped_processed_images

In [ ]:
THRESHOLD = 5.4
# loop over the input images
for imagePath in dataCropped[:1]:
  # load the image and process it
  image = cv2.imread(imagePath)

  input_image = np.asarray(tf.keras.preprocessing.image.load_img(imagePath, target_size=(224,224)))
  input_image = tf.keras.applications.mobilenet.preprocess_input(input_image)
  input_image = tf.convert_to_tensor(input_image, dtype=tf.float32)
  input_image = tf.expand_dims(input_image, axis = 0)
  
  interpreter.set_tensor(input_details[0]['index'], input_image)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])

  sample = calc_mean_score(output_data)
  
  text = "Not Blurry"
  color = (0,255,0)
  # if the focus measure is less than the supplied threshold,
  # then the image should be considered "blurry"
  if sample < THRESHOLD:
    text = "Blurry"
    color = (0,0,255)
  # show the image
  cv2.putText(image, "{}: {:.2f}".format(text, sample), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 3)
  cv2_imshow(image)
  key = cv2.waitKey(0)

  name = os.path.split(imagePath)[1]
  #print(name)
  
  #write the processed image to specific folder
  cv2.imwrite(('/content/quality_cropped_processed_images/'+ name), image)

In [ ]:
#zip folder
!zip -r /content/quality_cropped_processed_images.zip /content/quality_cropped_processed_images

## Use result from cropped to full image

In [ ]:
!mkdir cropped_quality_images

In [ ]:

def im_crop_center(img, w, h):
    img_width, img_height = img.size
    left, right = (img_width - w) / 2, (img_width + w) / 2
    top, bottom = (img_height - h) / 2, (img_height + h) / 2
    left, top = round(max(0, left)), round(max(0, top))
    right, bottom = round(min(img_width - 0, right)), round(min(img_height - 0, bottom))
    return img.crop((left, top, right, bottom))


for path in data:
  print(path)
  pil_image = Image.open(path)

  open_cv_image = np.array(im_crop_center(pil_image, 640, 480)) 
  # Convert RGB to BGR 
  open_cv_image = open_cv_image[:, :, ::-1].copy()
  name = os.path.split(path)[1]
  cv2.imwrite(('/content/cropped_quality_images/'+ name), open_cv_image)
  #cv2_imshow(open_cv_image)

In [ ]:
dataCropped = []
general_path_cropped = '/content/cropped_quality_images/'

for filename in os.listdir(general_path_cropped):
    if filename.endswith("jpg"): 
        # Your code comes here such as 
        dataCropped.append(general_path_cropped + filename)

print(dataCropped)

In [ ]:
!mkdir quality_cropped_to_full_processed_images

In [ ]:
THRESHOLD = 3.7
# loop over the input images
for imagePath in dataCropped[:10]:
  # load the image and process it
  name = os.path.split(imagePath)[1]
  #image = cv2.imread(imagePath)
  image = cv2.imread('/content/blur_images/' + name)

  input_image = np.asarray(tf.keras.preprocessing.image.load_img(imagePath, target_size=(224,224)))
  input_image = tf.keras.applications.mobilenet.preprocess_input(input_image)
  input_image = tf.convert_to_tensor(input_image, dtype=tf.float32)
  input_image = tf.expand_dims(input_image, axis = 0)
  
  interpreter.set_tensor(input_details[0]['index'], input_image)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])

  sample = calc_mean_score(output_data)
  
  text = "Not Blurry"
  color = (0,255,0)
  # if the focus measure is less than the supplied threshold,
  # then the image should be considered "blurry"
  if sample < THRESHOLD:
    text = "Blurry"
    color = (0,0,255)
  # show the image
  cv2.putText(image, "{}: {:.2f}".format(text, sample), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 3)
  print(name)
  cv2_imshow(image)
  key = cv2.waitKey(0)
  
  #write the processed image to specific folder
  cv2.imwrite(('/content/quality_cropped_to_full_processed_images/'+ name), image)

In [ ]:
#zip folder
!zip -r /content/quality_cropped_to_full_processed_images.zip /content/quality_cropped_to_full_processed_images